In [189]:
import pandas as pd
import gspread

import youtube_dl
ytdlopts = {
    'format': 'bestaudio/best',
    'outtmpl': 'downloads/%(extractor)s-%(id)s-%(title)s.%(ext)s',
    'restrictfilenames': True,
    'noplaylist': True,
    'nocheckcertificate': True,
    'ignoreerrors': False,
    'logtostderr': False,
    'quiet': True,
    'no_warnings': True,
    'default_search': 'auto',
    'source_address': '0.0.0.0'  # ipv6 addresses cause issues sometimes
}
ytdl = youtube_dl.YoutubeDL(ytdlopts)

In [190]:
def get_ytb_track_data(self, row):
    data = ytdl.extract_info(url=row.URL, download=False)
    if 'entries' in data:
        if len(data['entries']) == 1:  # for search single song
            data['duration'] = data['entries'][0]['duration']
            data['view_count'] = data['entries'][0]['view_count']
            data['categories'] = data['entries'][0]['categories']

    duration = get_readable_duration(data['duration']) if 'duration' in data else ""
    views = f"{data['view_count']:,}" if 'view_count' in data else ""
    categories = ', '.join(data['categories']) if 'categories' in data else ""

    return duration, views, categories

def get_readable_duration(duration):
    """Get duration in hours, minutes and seconds."""

    m, s = divmod(int(duration), 60)
    h, m = divmod(m, 60)

    if h:
        duration = f"{h}:{m:02d}:{s:02d}"
    else:
        duration = f"{m}:{s:02d}"

    return duration

In [191]:
# get pandas format
gc = gspread.service_account(filename='credentials.json')
sh = gc.open("Discord Music Log")
cmd_wks = sh.worksheet("Commands Log2")
cmd_df = pd.DataFrame(cmd_wks.get_all_records())
cmd_df['Date'] = pd.to_datetime(cmd_df['Date'])
now = pd.Timestamp.now()
sheet_name = "Track Log (year)"
offset = pd.DateOffset(years=1)

track_wks = sh.worksheet(sheet_name)
track_df = pd.DataFrame(track_wks.get_all_records())
if track_df.empty:
    track_df = pd.DataFrame(columns=[
        'First time requested', 'Last time requested',
        'Requests', 'Title', 'URL',
        'Duration', 'Views', 'Categories'
    ])
track_df.head(60)

,First time requested,Last time requested,Requests,Title,URL,Duration,Views,Categories


In [192]:
# filter months
if offset:
    timestamp = now - offset
    filter_ = cmd_df['Date'] >= timestamp
    filtered_cmd_df = cmd_df[filter_]
else:
    filtered_cmd_df = cmd_df
filtered_cmd_df


,Date,Author,Title,URL
3175,2021-05-27 12:16:17,Bogori,21st Century Schizoid Man (Radio Edit),https://www.youtube.com/watch?v=AWCV9LLTHx0
3176,2021-05-27 12:21:40,Freezpmark,Kingslayer「AMV」 Fate Series,https://www.youtube.com/watch?v=oxDKF_x3Efc
3177,2021-05-27 12:35:48,Bogori,NIVIRO - The Labyrinth [NCS Release],https://www.youtube.com/watch?v=4S0shEqJ8Vc
3178,2021-05-27 12:39:28,Bogori,Here Comes the Hotstepper (Evian version - Yuk...,https://www.youtube.com/watch?v=nlqNHp3qsdU
3179,2021-05-27 12:44:22,Freezpmark,Doki Doki Literature Club! OST - Doki Doki Lit...,https://www.youtube.com/watch?v=BFSWlDpA6C4
...,...,...,...,...
3537,2022-04-27 22:29:40,Freezpmark,Perturbator - Ghost Dancers Slay Together,https://www.youtube.com/watch?v=m8Ap8nVB2p8
3538,2022-04-27 22:29:50,Freezpmark,Vogel im Käfig,https://www.youtube.com/watch?v=H76_uW1Fnso
3539,2022-04-27 23:29:56,Freezpmark,Vogel im Käfig,https://www.youtube.com/watch?v=H76_uW1Fnso
3540,2022-04-27 23:29:57,Freezpmark,Crystal Waters - Gypsy Woman (She's Homeless) ...,https://www.youtube.com/watch?v=_KztNIg4cvE


In [193]:
# groupby titles
grouped_cmd_df = filtered_cmd_df.groupby(["Title", "URL"])["Date"]
function_list = [("First time requested", "min"), ("Last time requested", "max"), ("Requests", "count")]
grouped_cmd_df = grouped_cmd_df.agg(function_list).reset_index()
grouped_cmd_df

,Title,URL,First time requested,Last time requested,Requests
0,2022,https://www.youtube.com/playlist?list=PLgLNILF...,2022-04-27 22:29:14,2022-04-27 22:29:18,2
1,(ENG) Dreamcatcher(드림캐쳐) 'MAISON' MV,https://www.youtube.com/watch?v=z4t9LLq1Nk0,2022-04-17 12:46:46,2022-04-18 18:09:03,2
2,06 - Gakuen Tengoku (Full Hensei Ver.) ~ Hibik...,https://www.youtube.com/watch?v=0xj11x6eDuI,2022-04-27 14:37:06,2022-04-27 14:37:06,1
3,10 - RYDEEN ~ Hibike! Euphonium - [Zona Random],https://www.youtube.com/watch?v=_ic07gdsGYQ,2022-04-27 14:33:59,2022-04-27 14:33:59,1
4,21st Century Schizoid Man (Radio Edit),https://www.youtube.com/watch?v=AWCV9LLTHx0,2021-05-27 12:16:17,2021-05-27 12:16:17,1
...,...,...,...,...,...
241,「Creditless」Kaguya-sama: Love is War OP / Open...,https://www.youtube.com/watch?v=kkVNjjUgYdY,2021-08-20 15:47:05,2022-04-18 21:15:35,2
242,【東方ヴォーカルPV】Necromantic【暁Records公式】,https://www.youtube.com/watch?v=RtTYQuO1j6w,2022-04-17 13:54:30,2022-04-17 13:54:30,1
243,ろん -「おちゃめ機能」FUKKIRETA 【HD.256k.Kara】,https://www.youtube.com/watch?v=kuNixp-wvWM,2022-04-17 13:45:08,2022-04-17 13:45:08,1
244,喜劇,https://www.youtube.com/watch?v=URR_Nv-6MZs,2022-04-10 15:13:17,2022-04-10 15:13:17,1


In [195]:
# groupby titles
grouped_cmd_df = filtered_cmd_df.groupby(["URL", "Title"])["Date"]
function_list = [("First time requested", "min"), ("Last time requested", "max"), ("Requests", "count")]
grouped_cmd_df = grouped_cmd_df.agg(function_list).reset_index()

# merge with track_df, rearrange, clean data
# cols_to_use = track_df.columns.difference(grouped_cmd_df.columns)
# track_df = track_df.sort_values(["Title", "URL"])
track_df = track_df.drop(labels=["First time requested", "Last time requested", "Requests", "Title"], axis=1)
merged_df = pd.merge(grouped_cmd_df, track_df, on='URL', how="left")
merged_df
grouped_cmd_df
# merged_df = merged_df[[
#                 'First time requested', 'Last time requested',
#                 'Requests', 'Title', 'URL',
#                 'Duration', 'Views', 'Categories'
#             ]]


,URL,Title,First time requested,Last time requested,Requests
0,https://www.youtube.com/playlist?list=PLHqc-xB...,Dano Drevo a turnaj mekyho žbirku Full List,2022-04-01 13:44:33,2022-04-01 14:45:42,2
1,https://www.youtube.com/playlist?list=PLgLNILF...,2022,2022-04-27 22:29:14,2022-04-27 22:29:18,2
2,https://www.youtube.com/playlist?list=PLqynaP0...,Painkiller Black Edition OST (fight),2022-04-01 16:45:26,2022-04-01 16:45:26,1
3,https://www.youtube.com/watch?v=--9kqhzQ-8Q,H.Y.C.Y.BH? (Official Video),2022-04-02 15:44:23,2022-04-03 17:52:06,2
4,https://www.youtube.com/watch?v=-81PSoMeTUM,KAGUYA-SAMA WA KOKURASETAI TENSAITACHI NO REN'...,2022-04-10 15:01:03,2022-04-10 15:01:03,1
...,...,...,...,...,...
241,https://www.youtube.com/watch?v=zVch_PK7SOY,Pendulum - Tarantula,2022-04-18 20:12:26,2022-04-18 20:12:26,1
242,https://www.youtube.com/watch?v=zWaymcVmJ-A,Los Del Rio - Macarena (Bayside Boys Remix),2022-04-17 13:19:35,2022-04-17 13:19:35,1
243,https://www.youtube.com/watch?v=zZMg9ryeWOw,Mick Gordon - 02. Rip & Tear,2021-08-17 19:42:02,2022-04-03 19:06:09,2
244,https://www.youtube.com/watch?v=zroFzv7sFis,ətˈæk 0N tάɪtn,2022-04-18 19:40:18,2022-04-18 19:40:18,1


In [196]:
# track_df.reset_index(drop=True)
merged_df

,URL,Title,First time requested,Last time requested,Requests,Duration,Views,Categories
0,https://www.youtube.com/playlist?list=PLHqc-xB...,Dano Drevo a turnaj mekyho žbirku Full List,2022-04-01 13:44:33,2022-04-01 14:45:42,2,NaN,NaN,NaN
1,https://www.youtube.com/playlist?list=PLgLNILF...,2022,2022-04-27 22:29:14,2022-04-27 22:29:18,2,NaN,NaN,NaN
2,https://www.youtube.com/playlist?list=PLqynaP0...,Painkiller Black Edition OST (fight),2022-04-01 16:45:26,2022-04-01 16:45:26,1,NaN,NaN,NaN
3,https://www.youtube.com/watch?v=--9kqhzQ-8Q,H.Y.C.Y.BH? (Official Video),2022-04-02 15:44:23,2022-04-03 17:52:06,2,NaN,NaN,NaN
4,https://www.youtube.com/watch?v=-81PSoMeTUM,KAGUYA-SAMA WA KOKURASETAI TENSAITACHI NO REN'...,2022-04-10 15:01:03,2022-04-10 15:01:03,1,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
241,https://www.youtube.com/watch?v=zVch_PK7SOY,Pendulum - Tarantula,2022-04-18 20:12:26,2022-04-18 20:12:26,1,NaN,NaN,NaN
242,https://www.youtube.com/watch?v=zWaymcVmJ-A,Los Del Rio - Macarena (Bayside Boys Remix),2022-04-17 13:19:35,2022-04-17 13:19:35,1,NaN,NaN,NaN
243,https://www.youtube.com/watch?v=zZMg9ryeWOw,Mick Gordon - 02. Rip & Tear,2021-08-17 19:42:02,2022-04-03 19:06:09,2,NaN,NaN,NaN
244,https://www.youtube.com/watch?v=zroFzv7sFis,ətˈæk 0N tάɪtn,2022-04-18 19:40:18,2022-04-18 19:40:18,1,NaN,NaN,NaN


In [181]:
track_df

,Title,Duration,Views,Categories


In [5]:
for i, row in enumerate(merged_df.head(24).itertuples()):
    ytb_stats = row.Duration, row.Views, row.Categories
    if not all(ytb_stats):
        try:
            duration, views, categories = self.get_ytb_track_data(row)
        except Exception as e:
            print(f"{i}. error: {e}. (row: {row})")
            continue

        merged_df.at[row.Index, 'Duration'] = duration.replace(":", "︰")
        merged_df.at[row.Index, 'Views'] = views
        merged_df.at[row.Index, 'Categories'] = categories
        print(f"Updated {i} row.")

# save to gsheets
listed_table_result = [merged_df.columns.values.tolist()] + merged_df.values.tolist()  # first part is for header
track_wks.update(listed_table_result, value_input_option='USER_ENTERED')  # value_input_option='USER_ENTERED' / 'RAW'


,First time requested,Last time requested,Requests,Title,URL,Duration,Views,Categories
826,2020-09-17 21:10:04,2020-09-17 21:10:04,1,Kopytovci - Dojit HD,https://www.youtube.com/watch?v=0khMME8R2Fc,2︰16,4336679,Music
967,2019-04-25 20:38:53,2019-04-25 20:38:53,1,MinAh - Cry Me Out / I am a Woman Too [Yu Huiy...,https://www.youtube.com/watch?v=RxfoFcNYDc4,4︰31,2191096,Music
966,2020-03-03 21:37:43,2020-03-03 21:37:43,1,Milky Chance - Stolen Dance (Official Video),https://www.youtube.com/watch?v=iX-QaNzd-0Y,4︰40,2193950,Music
965,2019-04-15 20:54:43,2019-04-15 20:54:43,1,Miley Cyrus - The Backyard Sessions - 'Jolene',https://www.youtube.com/watch?v=wOwblaKmyVw,3︰19,2202357,Music
964,2019-10-11 22:20:55,2019-10-11 22:20:55,1,Miho Hazama: Dancer in Nowhere feat. Nate Wood,https://www.youtube.com/watch?v=q-z-2SXmlnE,10︰31,2204418,Music
...,...,...,...,...,...,...,...,...
445,2020-09-19 22:09:42,2022-04-03 18:58:14,25,Dreamcatcher(드림캐쳐) 'Scream' MV,https://www.youtube.com/watch?v=FKlGHHhTOsQ,3︰58,36425860,Music
393,2019-03-31 19:54:25,2022-04-10 16:03:27,26,Devil May Cry 5 - Devil Trigger (Ali Edwards) ...,https://www.youtube.com/watch?v=XpPaTM48h74,3︰43,50946803,Music
34,2019-03-16 18:48:29,2021-08-20 15:23:27,26,4MINUTE(포미닛) - 싫어(Hate) MV,https://www.youtube.com/watch?v=D_F9HUTYnl0,3︰47,1210343404,Music
1565,2019-03-22 21:26:52,2022-04-10 16:13:09,29,[Full Song/Official Lyrics] Devil Trigger - Ne...,https://www.youtube.com/watch?v=-WpnPSChVRQ,3︰55,23780,Music


In [ ]:
# pd.set_option('display.max_rows', 100)

# grouped_cmd_df[['Duration', 'Views', 'Categories']] = 0
# grouped_cmd_df = grouped_cmd_df[[
#     'First time requested', 'Last time requested',
#     'Requests', 'Title', 'URL',
#     'Duration', 'Views', 'Categories'
# ]]
# grouped_cmd_df

# track_df = track_df.sort_values(["Title", "URL"]).reset_index(drop=True)
# track_df
# rows = (grouped_cmd_df[['Title']] == track_df[['Title']]).all(axis=1)
# grouped_cmd_df.loc[rows, 'Views'] = track_df.loc[rows, 'Views']
# grouped_cmd_df

# # mask = grouped_cmd_df[['Title']].isin(track_df[['Title']]).all(1)
# # grouped_cmd_df.Views = grouped_cmd_df.Views.mask(mask).combine_first(track_df.Views)
# # grouped_cmd_df.head(60)

# rows = (grouped_cmd_df[['Title','URL']] == track_df[['Title','URL']]).all(axis=1)
# grouped_cmd_df.loc[rows,'Views'] = track_df.loc[rows,'Views']
# #grouped_cmd_df[['Duration', 'Views', 'Categories']] = track_df[['Duration', 'Views', 'Categories']]
# grouped_cmd_df

